**Library Imports**

In [1]:
import pandas as pd
import io
import requests
import numpy as np
import os
import seaborn as sns

from scipy.stats import zscore

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelEncoder
from IPython.display import display



import tensorflow as tf

Panda Preferences

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

Read CSV

In [3]:
df = pd.read_csv("flows_cleaned/monthly/202208_combined.csv")
display(df)

,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,fwd_header_size_max,bwd_header_size_tot,bwd_header_size_min,bwd_header_size_max,flow_FIN_flag_count,flow_SYN_flag_count,flow_RST_flag_count,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_URG_flag_count,bwd_URG_flag_count,flow_CWR_flag_count,flow_ECE_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.min,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,type
0,CALYWXjY1simu8JKl,192.168.1.10,51301,192.168.1.5,1880,tcp,-,0.005211,0,0,SF,-,-,0,FfA,2,80,1,40,-,0.005211,2,1,0,0,383.795031,191.897516,575.692547,0.500000,40,20,20,20,20,20,2,0,0,0,0,3,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.699996e+04,1.699996e+04,1.699996e+04,1.699996e+04,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5211.114883,1.178885e+04,1.699996e+04,8.499980e+03,4.651158e+03,0.000000,2.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16999.959946,1.699996e+04,1.699996e+04,1.699996e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,2144,64065,2143,64065,normal
1,Cyy6gA1iMi44yLoJn1,192.168.1.10,53204,192.168.1.5,1880,tcp,http,5.221258,181,0,SF,-,-,0,ShADaFf,6,425,4,164,-,5.221258,6,4,2,0,1.149148,0.766099,1.915247,0.666667,124,20,24,84,20,24,2,2,0,2,0,9,0,0,0,0,0.0,175.0,181.0,30.166667,70.994131,0.0,0.0,0.0,0.0,0.0,0.0,175.0,181.0,18.100000,55.161279,1.116037e+03,5.204530e+06,5.229443e+06,1.045889e+06,2.324755e+06,1052.141190,5.205845e+06,5.220774e+06,1.740258e+06,3.001293e+06,77.009201,5.204453e+06,5.229443e+06,5.810492e+05,1.733781e+06,34.665976,3.0,2.000000,90.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9577.035904,1.541305e+04,2.499008e+04,1.249504e+04,4.126682e+03,5.204453e+06,5.204453e+06,5.204453e+06,5.204453e+06,0.0,2144,64240,2143,64065,normal
2,C7Nd9d2f2bFQqbEvB,192.168.1.10,58526,192.168.1.5,1880,tcp,http,5.222343,181,0,SF,-,-,0,ShADaFf,6,425,4,164,-,5.222343,6,4,2,0,1.148910,0.765940,1.914849,0.666667,124,20,24,84,20,24,2,2,0,2,0,9,0,0,0,0,0.0,175.0,181.0,30.166667,70.994131,0.0,0.0,0.0,0.0,0.0,0.0,175.0,181.0,18.100000,55.161279,4.990101e+02,5.204551e+06,5.231287e+06,1.046257e+06,2.324562e+06,151.872635,5.204884e+06,5.222199e+06,1.740733e+06,3.000055e+06,44.822693,5.204506e+06,5.231287e+06,5.812541e+05,1.733726e+06,34.658773,3.0,2.000000,90.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9321.928024,1.745892e+04,2.678084e+04,1.339042e+04,5.753719e+03,5.204506e+06,5.204506e+06,5.204506e+06,5.204506e+06,0.0,2144,64240,2143,64065,normal
3,CCGYam1HbSYMX4s4dg,192.168.1.10,63975,192.168.1.5,1880,tcp,http,5.215404,182,0,SF,-,-,0,ShADaFf,6,426,4,164,-,5.215404,6,4,2,0,1.150438,0.766959,1.917397,0.666667,124,20,24,84,20,24,2,2,0,2,0,9,0,0,0,0,0.0,176.0,182.0,30.333333,71.402147,0.0,0.0,0.0,0.0,0.0,0.0,176.0,182.0,18.200000,55.477323,2.257824e+02,5.203449e+06,5.220297e+06,1.044059e+06,2.325171e+06,140.905380,5.203837e+06,5.215294e+06,1.7

Fix Dataframe Mixed Types

In [4]:
def fix_data_mixed_types(df, mixed_columns):    
    # Loop over the mixed columns and clean them
    for col_idx in mixed_columns:
        col_name = df.columns[col_idx]
        dtype_before = df[col_name].dtype
        
        # Convert non-numeric values to NaN and replace with mean
        df[col_name] = pd.to_numeric(df[col_name], errors='coerce')
        mean = df[col_name].mean()
        df[col_name].fillna(mean, inplace=True)
        
        dtype_after = df[col_name].dtype
        print(f"[INFO] Before: {dtype_before} | After: {dtype_after}")
        
    return df

In [5]:
fix_data_mixed_types(df, [7, 8, 9, 17])

[INFO] Before: object | After: float64
[INFO] Before: object | After: float64
[INFO] Before: object | After: float64
[INFO] Before: int64 | After: int64


,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,fwd_header_size_max,bwd_header_size_tot,bwd_header_size_min,bwd_header_size_max,flow_FIN_flag_count,flow_SYN_flag_count,flow_RST_flag_count,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_URG_flag_count,bwd_URG_flag_count,flow_CWR_flag_count,flow_ECE_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.min,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,type
0,CALYWXjY1simu8JKl,192.168.1.10,51301,192.168.1.5,1880,tcp,-,0.005211,0.0,0.0,SF,-,-,0,FfA,2,80,1,40,-,0.005211,2,1,0,0,383.795031,191.897516,575.692547,0.500000,40,20,20,20,20,20,2,0,0,0,0,3,0,0,0,0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.699996e+04,1.699996e+04,1.699996e+04,1.699996e+04,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5211.114883,1.178885e+04,1.699996e+04,8.499980e+03,4.651158e+03,0.000000,2.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16999.959946,1.699996e+04,1.699996e+04,1.699996e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,2144,64065,2143,64065,normal
1,Cyy6gA1iMi44yLoJn1,192.168.1.10,53204,192.168.1.5,1880,tcp,http,5.221258,181.0,0.0,SF,-,-,0,ShADaFf,6,425,4,164,-,5.221258,6,4,2,0,1.149148,0.766099,1.915247,0.666667,124,20,24,84,20,24,2,2,0,2,0,9,0,0,0,0,0.0,175.0,181.0,30.166667,70.994131,0.0,0.0,0.0,0.0,0.0,0.0,175.0,181.0,18.100000,55.161279,1.116037e+03,5.204530e+06,5.229443e+06,1.045889e+06,2.324755e+06,1052.141190,5.205845e+06,5.220774e+06,1.740258e+06,3.001293e+06,77.009201,5.204453e+06,5.229443e+06,5.810492e+05,1.733781e+06,34.665976,3.0,2.000000,90.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9577.035904,1.541305e+04,2.499008e+04,1.249504e+04,4.126682e+03,5.204453e+06,5.204453e+06,5.204453e+06,5.204453e+06,0.0,2144,64240,2143,64065,normal
2,C7Nd9d2f2bFQqbEvB,192.168.1.10,58526,192.168.1.5,1880,tcp,http,5.222343,181.0,0.0,SF,-,-,0,ShADaFf,6,425,4,164,-,5.222343,6,4,2,0,1.148910,0.765940,1.914849,0.666667,124,20,24,84,20,24,2,2,0,2,0,9,0,0,0,0,0.0,175.0,181.0,30.166667,70.994131,0.0,0.0,0.0,0.0,0.0,0.0,175.0,181.0,18.100000,55.161279,4.990101e+02,5.204551e+06,5.231287e+06,1.046257e+06,2.324562e+06,151.872635,5.204884e+06,5.222199e+06,1.740733e+06,3.000055e+06,44.822693,5.204506e+06,5.231287e+06,5.812541e+05,1.733726e+06,34.658773,3.0,2.000000,90.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9321.928024,1.745892e+04,2.678084e+04,1.339042e+04,5.753719e+03,5.204506e+06,5.204506e+06,5.204506e+06,5.204506e+06,0.0,2144,64240,2143,64065,normal
3,CCGYam1HbSYMX4s4dg,192.168.1.10,63975,192.168.1.5,1880,tcp,http,5.215404,182.0,0.0,SF,-,-,0,ShADaFf,6,426,4,164,-,5.215404,6,4,2,0,1.150438,0.766959,1.917397,0.666667,124,20,24,84,20,24,2,2,0,2,0,9,0,0,0,0,0.0,176.0,182.0,30.333333,71.402147,0.0,0.0,0.0,0.0,0.0,0.0,176.0,182.0,18.200000,55.477323,2.257824e+02,5.203449e+06,5.220297e+06,1.044059e+06,2.325171e+06,140.905380,5.203837e+06,

-----------------------------------------------------------

**DF Statistics and Info**

In [6]:
def displayInformationDataFrame(df_cop):
    summary_df = pd.DataFrame(columns=['Data Type', 'Column Name', 'Unique Values'])
    # Iterate through the columns of the original dataframe
    for col in df_cop.columns:
        # Get the data type of the column
        dtype = df_cop[col].dtype
        # Get the column name
        col_name = col
        # Get the unique values of the column
        unique_values = df_cop[col].unique()
        # Append a new row to the summary dataframe
        summary_df = summary_df.append({'Data Type': dtype, 'Column Name': col_name, 'Unique Values': unique_values}, ignore_index=True)
    # display the summary_df
    pd.options.display.max_rows = None
    pd.options.display.max_columns = None
    return display(summary_df)

In [7]:
displayInformationDataFrame(df)

,Data Type,Column Name,Unique Values
0,object,uid,"[CALYWXjY1simu8JKl, Cyy6gA1iMi44yLoJn1, C7Nd9d..."
1,object,id.orig_h,"[192.168.1.10, 192.168.1.8, 192.168.1.5, fe80:..."
2,int64,id.orig_p,"[51301, 53204, 58526, 63975, 60977, 64799, 550..."
3,object,id.resp_h,"[192.168.1.5, 185.125.190.58, ff02::fb, 255.25..."
4,int64,id.resp_p,"[1880, 137, 123, 0, 5353, 67, 138, 80, 1900, 3..."
5,object,proto,"[tcp, udp, icmp]"
6,object,service,"[-, http, dns, ntp, dhcp, ssl]"
7,float64,duration,"[0.005211, 5.221258, 5.222343, 5.215404, 5.218..."
8,float64,orig_bytes,"[0.0, 181.0, 182.0, 353.0, 150.0, 48.0, 23.0, ..."
9,float64,resp_bytes,"[0.0, 284.0, 48.0, 23.0, 57.430948761691084, 7..."


In [8]:
print(f"Number of NaN values:  {df.isnull().sum().sum()}")

Number of NaN values:  0


In [9]:
df.describe(exclude=[object]).loc[['mean','std','min','25%','50%','75%','max']].T

,mean,std,min,25%,50%,75%,max
id.orig_p,3.182166e+04,1.957864e+04,0.000000,14268.000000,3.349700e+04,4.874800e+04,6.553500e+04
id.resp_p,4.360582e+03,9.700334e+03,0.000000,0.000000,1.883000e+03,5.683000e+03,6.553300e+04
duration,2.717686e+01,3.855514e+01,0.000016,24.711913,2.717686e+01,2.717686e+01,7.446048e+03
orig_bytes,7.599113e+06,5.585659e+07,0.000000,0.000000,7.599113e+06,7.599113e+06,2.061860e+09
resp_bytes,5.743095e+01,4.583919e+03,0.000000,0.000000,5.743095e+01,5.743095e+01,1.708187e+06
missed_bytes,2.885237e+06,5.539349e+07,0.000000,0.000000,0.000000e+00,0.000000e+00,2.061860e+09
orig_pkts,5.115807e+00,1.605311e+03,0.000000,1.000000,1.000000e+00,1.000000e+00,9.991690e+05
orig_ip_bytes,5.306655e+02,4.545089e+04,0.000000,40.000000,4.000000e+01,4.400000e+01,2.797673e+07
resp_pkts,1.662818e+00,2.127527e+02,0.000000,0.000000,0.000000e+00,0.000000e+00,1.219070e+05
resp_ip_bytes,9.882877e+01,8.524074e+03,0.000000,0.000000,0.000000e+00,0.000000e+00,3.413396e+06


--------------------------------------------

**Pre-processing and Data Encoding**

Delete Insignificant Columns from the Dataframe

In [10]:
def delete_columns(df, cols):
    for col in cols:
        df.drop(col, axis = 1, inplace = True)
        print(f'[REMOVED] {col}')
    
    return df

In [11]:
cols_to_del = [
    'uid',
    'id.orig_h',
    'id.orig_p',
    'id.resp_h',
    'id.resp_p',
    'active.min'
    ]

df = delete_columns(df,cols_to_del)

[REMOVED] uid
[REMOVED] id.orig_h
[REMOVED] id.orig_p
[REMOVED] id.resp_h
[REMOVED] id.resp_p
[REMOVED] active.min


Normalize, Z-score

In [12]:
def zscore_normalization(df, cols):
    # Standardize the selected columns
    for col in cols:
        if col not in df.columns:
            print(f"[WARNING] {col} not found in DataFrame.")
            continue
        df[col] = zscore(df[col])
    
    print("[DONE] Z-score Normalization")
    print("[INFO] Current Fields in the DataFrame:")
    display(df)
    
    return df

In [13]:
cols_to_zscore = [
    'flow_duration', 'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot',
    'bwd_data_pkts_tot', 'fwd_pkts_per_sec', 'bwd_pkts_per_sec',
    'flow_pkts_per_sec', 'down_up_ratio', 'fwd_header_size_tot',
    'fwd_header_size_min', 'fwd_header_size_max', 'bwd_header_size_tot',
    'bwd_header_size_min', 'bwd_header_size_max', 'flow_FIN_flag_count',
    'flow_SYN_flag_count', 'flow_RST_flag_count', 'fwd_PSH_flag_count',
    'bwd_PSH_flag_count', 'flow_ACK_flag_count', 'fwd_pkts_payload.min',
    'fwd_pkts_payload.max', 'fwd_pkts_payload.tot', 'fwd_pkts_payload.avg',
    'fwd_pkts_payload.std', 'bwd_pkts_payload.min', 'bwd_pkts_payload.max',
    'bwd_pkts_payload.tot', 'bwd_pkts_payload.avg', 'bwd_pkts_payload.std',
    'flow_pkts_payload.min', 'flow_pkts_payload.max', 'flow_pkts_payload.tot',
    'flow_pkts_payload.avg', 'flow_pkts_payload.std', 'fwd_iat.min',
    'fwd_iat.max', 'fwd_iat.tot', 'fwd_iat.avg', 'fwd_iat.std',
    'bwd_iat.min', 'bwd_iat.max', 'bwd_iat.tot', 'bwd_iat.avg', 'bwd_iat.std',
    'flow_iat.min', 'flow_iat.max', 'flow_iat.tot', 'flow_iat.avg',
    'flow_iat.std', 'payload_bytes_per_second', 'fwd_subflow_pkts',
    'bwd_subflow_pkts', 'fwd_subflow_bytes', 'bwd_subflow_bytes',
    'fwd_bulk_bytes', 'bwd_bulk_bytes', 'fwd_bulk_packets', 'bwd_bulk_packets',
    'fwd_bulk_rate', 'bwd_bulk_rate', 'active.max', 'active.tot',
    'active.avg', 'active.std', 'idle.min', 'idle.max', 'idle.tot',
    'idle.avg', 'idle.std', 'fwd_init_window_size', 'bwd_init_window_size',
    'fwd_last_window_size', 'bwd_last_window_size'
]

df = zscore_normalization(df, cols_to_zscore)

[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


Objects to int

In [14]:
def service_to_int(service):
    mapping = {'-': 0, 'http': 1, 'dns': 2, 'ntp': 3, 'dtls': 4, 'dhcp': 5, 'radios': 6, 'ssl': 7}
    return mapping.get(service, 8)

In [15]:
df['service'] = df.apply(lambda row: service_to_int(row['service']), axis=1)

In [16]:
def proto_to_int(proto):
    mapping = {'tcp': 6, 'udp': 17, 'icmp': 1}
    return mapping.get(proto, 0)

In [17]:
df['proto'] = df.apply(lambda row: proto_to_int(row['proto']), axis=1)

In [18]:
#Attack Identification

# 1 - normal
# 2 - arp_spoofing
# 3 - http_botnet
# 4 - http_flood_node_red
# 5 - icmp_flood
# 6 - mqtt_flood
# 7 - port_scanning
# 8 - tcp_flood
# 9 - udp_flood

def attack_to_int(attack):
    mapping = {'normal': 1, 'arp_spoofing': 2, 'http_botnet':3, 'http_flood_node_red':4, 'icmp_flood':5, 'mqtt_flood':6, 'port_scanning':7,'tcp_flood':8,'udp_flood':9}
    return mapping.get(attack, 0)

In [19]:
df['type'] = df.apply(lambda row: attack_to_int(row['type']), axis=1)

In [20]:
def conn_state_to_int(conn_state):
    mapping = {'S0': 0, 'S1': 1, 'SF': 2, 'REJ': 3, 'S2': 4, 'S3': 5, 'RSTO': 6, 'RSTR': 7, 'RSTOS0': 8, 'RSTRH': 9, 'SH': 10, 'SHR': 11, 'OTH': 12}
    return mapping.get(conn_state, 13)

In [21]:
df['conn_state'] = df.apply(lambda row: conn_state_to_int(row['conn_state']), axis=1)

Normalizing Duration Column - Using The min-max feature scaling to keep the outliers

In [22]:
df['duration'] = df['duration'].astype(int)

In [23]:
def mqtt_flood_missed_bytes(missed_bytes):
    if missed_bytes < 1:
        return 0
    else:
        return 1

In [24]:
df['missed_bytes'] = df.apply(lambda row: mqtt_flood_missed_bytes(row['missed_bytes']), axis=1)
df.groupby(['missed_bytes'])['missed_bytes'].count().reset_index(name='count').sort_values(['count'], ascending=True)

,missed_bytes,count
1,1,1554
0,0,386919


In [25]:
def history_to_int(history):
    hist_list = list(history)
    sum = 0
    mapping = {'s': 1, 'S': 2, 'h': 3, 'H': 4, 'a': 5, 'A': 6, 'd': 7, 'D': 8, 'f': 9, 'F': 10, 'r': 11, 'R': 12, 'c': 13, 'C': 14, 'g': 15, 'G': 16, 't': 17, 'T': 18, 'w': 19, 'W': 20, 'i': 21, 'I': 22, 'q': 23, 'Q': 24, '^': 25}
    for char in hist_list:
        sum = sum + mapping.get(char,0)
    return sum

In [26]:
df['history'] = df.apply(lambda row: history_to_int(row['history']), axis=1)

Min Max Normalization

In [27]:
def min_max_feature(df, column):
    return (df[column] - df[column].min()) / (df[column].max() - df[column].min())

In [28]:
df['orig_bytes'] = min_max_feature(df,'orig_bytes')
df['resp_bytes'] = min_max_feature(df,'orig_bytes')

In [29]:
# orig_pkts doesn't need normalization
# resp_pkts doesn't need normalization, but has outliers

df['orig_ip_bytes'] = (df['orig_ip_bytes'] / 10).astype(int)
df['resp_ip_bytes'] = (df['resp_ip_bytes'] / 10).astype(int)
# Should i leave the outliers or remove them

In [ ]:
display(df)

-----------------------

**Plots**

In [ ]:
df.hist()
pyplot.show()

Bar Plot of Type Attack by Popularity of 5

In [ ]:
def type_attack_bar_plot_by_popularity(df,column):
    type_df = df[['type', column]]
    grouped = type_df.groupby(['type', column]).size().reset_index(name='counts')
    grouped_top5 = grouped.groupby('type').apply(lambda x: x.nlargest(5, 'counts'))

    plt.figure(figsize=(16, 12))
    sns.barplot(x='type', y=column, data=grouped_top5, hue=column, palette='viridis')
    plt.xlabel('Attack Type')
    plt.ylabel(column)
    plt.title(f'Top 5 Most Common {column} by Attack Type')
    plt.show()

In [ ]:
type_attack_bar_plot_by_popularity(df, 'conn_state')

In [ ]:
# select only the type_of_attack and duration columns
attacks = df.loc[:, ['type', 'duration']]

# group the data by type of attack and calculate the mean duration
mean_durations = attacks.groupby('type').mean()

# create a new figure with the desired size
fig = plt.figure(figsize=(10, 6))

# create a bar plot of the mean durations
ax = mean_durations.plot(kind='bar', rot=0)

# set the labels and title
ax.set_xlabel('Type of Attack')
ax.set_ylabel('Duration')
ax.set_title('Mean Duration of Attacks by Type')

# show the plot
plt.show()

In [ ]:
def heatmap(df,size=40):
    corr = df.corr().round(2)
    plt.figure(figsize=(size, size))
    sns.heatmap(corr, cmap="Blues", annot=True)
    plt.show()

In [ ]:
heatmap(df)

-------------------------------

Testing

In [ ]:
col_delete = ['local_orig','local_resp','tunnel_parents','fwd_URG_flag_count','bwd_URG_flag_count','flow_CWR_flag_count','flow_ECE_flag_count']
#col_delete = ['proto','fwd_init_window_size','fwd_header_size_max']
#col_delete = ['active.max','active.tot','active.avg','active.std','idle.min','idle.max','idle.tot','idle.avg','idle.std','fwd_init_window_size','bwd_init_window_size','fwd_last_window_size','bwd_last_window_size']
delete_columns(df,col_delete)

---------------------------------------

**Create Model & Train Model**

In [ ]:
df.groupby('type')['type'].count()

In [ ]:
# Convert to numpy - Classification
x_columns = df.columns.drop('type')
x = df[x_columns].values
dummies = pd.get_dummies(df['type']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

In [ ]:
# Shuffle the training set
#df = df.reindex(np.random.permutation(df.index))

# Create a test/train split.  20% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [ ]:
# Create neural net
model = Sequential()
model.add(Dense(3, input_dim=x.shape[1], activation='relu'))
model.add(Dense(12, input_dim=x.shape[1], activation='relu'))
model.add(Dense(9, input_dim=x.shape[1], activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto',
                           restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),
          callbacks=[monitor],verbose=2,epochs=90)

In [171]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.839976832486003


In [ ]:
def compute_metrics(pred, y_test):
    predict_classes = np.argmax(pred, axis = 1)
    expected_classes = np.argmax(y_test, axis = 1)
    
    correct = metrics.accuracy_score(expected_classes, predict_classes)
    print(f"Accuracy: {correct}")
    
    recall = metrics.recall_score(expected_classes, predict_classes, average = 'weighted')    
    print(f"Recall: {recall}")
       
    precision = metrics.precision_score(expected_classes, predict_classes, average = 'weighted')
    print(f"Precision: {precision}")
    
    f1score = metrics.f1_score(expected_classes, predict_classes, average = 'weighted')
    print(f"F1Score: {f1score}")

In [ ]:
pred = model.predict(x_test)
compute_metrics(pred, y_test)

-------------------------------------

**Result Plots**

In [ ]:
cm = confusion_matrix(y_eval, pred)
print(cm)

le = LabelEncoder()
le.fit(df["type"].values)
original_labels_list = le.classes_
fig,ax = plt.subplots(figsize=(20, 20))
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cm,display_labels= original_labels_list)

In [ ]:
# Confusion Matrix
predict_classes = np.argmax(pred, axis = 1)
expected_classes = np.argmax(y_test, axis = 1)    
    
cm = confusion_matrix(expected_classes, predict_classes)
cmd = ConfusionMatrixDisplay(cm)

# Plot size
fig, ax = plt.subplots(figsize = (10, 10))

cmd.plot(ax = ax)

#Attack Identification

# 1 - normal
# 2 - arp_spoofing
# 3 - http_botnet
# 4 - http_flood_node_red
# 5 - icmp_flood
# 6 - mqtt_flood
# 7 - port_scanning
# 8 - tcp_flood
# 9 - udp_flood

In [ ]:
# Usage of ExtraTreesClassifier for feature selection
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5, criterion ='entropy', max_features = 2)
extra_tree_forest.fit(x, y)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in  extra_tree_forest.estimators_], axis = 0)

print(f'[DONE] Extra Trees Classifier')

In [ ]:
# Plor for the ExtraTreesClassifier output
plot.bar(x_columns, feature_importance_normalized)
plot.xlabel('Feature Labels')
plot.ylabel('Feature Importances')
plot.title('Comparison of different feature importances in the current dataset')
plot.xticks(rotation = 90)

# Plot size
plot.rcParams["figure.figsize"] = (70, 40)

plot.show()